In [397]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import re

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [432]:
class utilities:
    
    #def 
    
    def wait(self, seconds):

        s = np.random.normal(0, 1)
        time.sleep(seconds*abs(s))
    
    def get_element_by_path(self, driver, xpath):
        try:
            return driver.find_element_by_xpath(xpath)
        except Exception as e:
            print(e)
            return None

In [433]:
class IndeedItemParser:
    def __init__(self):
        self.driverPath = "C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe"
        self.utilities = utilities();
        
    def _get_title(self, driver):
        try:
            title = driver.find_element_by_xpath("//*[@class='jobsearch-DesktopStickyContainer']//h3")
        except Exception as e:
            print(e)
        return title.text
    
    def _get_name(self, driver):
        
        try:    
            name = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[1]")
            return name.text
        except Exception as e:
            print(e)
        
    
    def _get_address(self,driver):
         try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[3]")
            if address.text == "-":
                address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[4]")
            return address.text
         except Exception as e:
            address = driver.find_element_by_xpath("//span[@class='jobsearch-JobMetadataHeader-iconLabel'][1]")
            return address.text
    
    def _get_salaire(self,driver, description):
        #jobsearch-JobMetadataHeader-item 
        """
        pattern = r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)?[\s+]?|(.*)?[\s+]?(par|\/)?[\s+]?(an|mois|jour|heure)'
        text = description
        try:
            text = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-JobMetadataHeader-item')]//span[1]").text
            print("first :", text)
            if text == "":
                raise Exception("empty")
        except Exception as e:
            print(e)
            try:   
                text = driver.find_element_by_xpath("//*[contains(@class,'icl-IconFunctional--salary')]//*[contains(@class,'jobsearch-JobMetadataHeader-iconLabel')]").text
                print("second : ",text)
            except Exception as e:
                print(e)
                print("troisième")
                
        outer = re.compile(pattern)
        m = outer.search(text)
        if m is not None:
              return m.group(1)
        return "-"
        """
        return "-"
            
    
    def _get_description(self,driver):
        try:
            #jobDescriptionText
            e_description = driver.find_element_by_id("jobDescriptionText")
            return e_description.text
        except Exception as e:
            print(e)
    
    def _get_date(self,driver,url,name):
       # print(url)
        try:
            date_str = driver.find_element_by_xpath("//*[@class='jobsearch-JobMetadataFooter']")
            date_str_full = date_str.text
            date_tbl = date_str_full.split(" ")
            count_str = date_tbl[4]
            label = date_tbl[5]
            
            if name in date_str_full:
                date_str_full = date_str_full.replace(name, "")
                date_tbl = date_str_full.split(" ")
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            if count_str == "a" :
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            #print("date_str", date_tbl)
            date = datetime.now()
            
            if count_str == "30+":
                return date - timedelta(days=30)
            
            count = int(count_str)
            if "jour" in label:
                date = date - timedelta(days=count)
            elif "heur" in label:
                date = date - timedelta(hours=count)
            return date;
        except Exception as e:
            print(e)
    
    
    def parse(self,url):
        driver = webdriver.Chrome(self.driverPath)
        driver.get(url)
        driver.maximize_window()
        
        source = driver.page_source
        title = self._get_title(driver)
        name = self._get_name(driver)
        address = self._get_address(driver)
        date = self._get_date(driver, url,name)
        description = self._get_description(driver)
        salaire = self._get_salaire(driver,description)
        
        driver.quit()
        
        return title, name, address, date, salaire, description, source

In [400]:
driver = webdriver.Chrome("C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe")
item_link ="https://www.indeed.fr/voir-emploi?cmp=Agence-ULTEAM&t=Chef+%C3%89quipe+Courrier&jk=c3b6cc0a43b192f4&sjdu=QwrRXKrqZ3CNX5W-O9jEvcttsaZdmkbfXA_O1RUuLLVt7V37OcJjVeywo21sCKKVZsKjH1QMOcrJLxLEe3LwbneMTtyfmo-r9mk-65EMnF_xE1dLTd8uf9wEDXC-y3DjRkj44UiUNV_6R9APTWrRDA&tk=1dm9ttu0492gm801&adid=316016867&vjs=3" 
driver.get(item_link)
source = driver.page_source
source_file = 'source.page.{0}.html'.format(item_link.replace("/","_").replace(":","_").replace(".","_").replace("-","_").replace("+","_").replace("?","_").replace("%","_"))
if not os.path.exists("source_file.html"):
    file = open("source_file.html", "w") 
    file.write(source) 
    file.close() 

In [395]:
#item_parser = IndeedItemParser()
#title, name, address, date, salaire,description,source = item_parser.parse("https://www.indeed.fr/voir-emploi?cmp=ECO--SI&t=Technicien+Support+Informatique&jk=dc1a032be2312c84&sjdu=QwrRXKrqZ3CNX5W-O9jEvX5ViC7ZmtCbUlpLMENSqzeEixTsazLVFu1bx1PMxHpgK0V934oVal0W7yYsbyobejEeYX4jvfhEEiUY8zoCrBk&tk=1dm9ttu0492gm801&adid=314524947&vjs=3")
#print(source)

In [434]:
import os.path
from os import path
import json

class IndeedParserStateObject(object):
    def __init__(self, data):
        self.__dict__ = json.loads(data)
        
class IndeedParserStateHandler:
    
    def __init__(self):
        self.filename = "indeed.parser.state.json"
        self.object = self._load_file()
        print(type(self.object))
        #self.object = IndeedParserStateObject(self.data)
    
    def save_state(self, job, location, page_request="", page_item_index=-1):
        self.object["job"] = job
        self.object["location"] = location
        self.object["page_request"] = page_request
        self.object["page_item_index"]  = page_item_index
        #self.data  = json.dumps(self.object)
        
        #data = json.dumps(self.object)
        self._save_file(self.object)
        #self.object = IndeedParserStateObject(self.data)
        
    def is_current_job(self, job):
        if (self.object["job"] == "") | (self.object["job"] == job):
            return True
        return False
    
    def is_current_location(self, location):
        if (self.object["location"] == "") | (self.object["location"] == location):
            return True
        return False
    
    def is_current_page_request(self, page_request):
        if (self.object["page_request"] == "") | (self.object["page_request"] == page_request):
            return True
        return False
    
    def _save_file(self, data):
         with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
                
    def _load_file(self):
        
        if path.exists(self.filename) == True:
            with open(self.filename) as json_file:
                data = json.load(json_file)
        else:
            data = {
                     "job": "",
                     "location":"",
                     "page_request":"",
                     "page_item_index":-1,
                    }
            with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
        
        return data


In [435]:
from datetime import datetime, timedelta
import os.path
import os
class IndeedPaser:
    def __init__(self):
        self.website = "https://www.indeed.fr"
        self.driverPath = "C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe"
        #self.dataset = pd.DataFrame(columns=['URL', 'Titre','Nom entreprise','Adresse','Date de publication', 'description'])
        
        if os.path.exists('indeed.csv') == True:
            self.dataset = pd.read_csv("indeed.csv")
        else:
            #URL,Titre,Nom entreprise,Adresse,Date de publication,salaire, description
            self.dataset = pd.DataFrame(columns=['URL','Titre','Nom entreprise','Adresse','Date de publication','salaire','description'])
        
        self.jobs = ["data scientist", "développeur", "data analyst", "business intelligence"]
        self.locations = ["Paris","Lyon", "Toulouse", "Nantes", "Bordeaux"]
        #self.locations = ["Paris", "Lyon", "Toulouse", "Nantes", "Bordeaux"]
        self.utilities =  utilities()
        self.indeed_item_parser = IndeedItemParser()
        self.parser_state = IndeedParserStateHandler()
    
    def _get_pages_counts(self,driver):
        #searchCountPages
        searchCountPages_elt = driver.find_element_by_id("searchCountPages")
        searchCountPages = searchCountPages_elt.text.split()
        if len(searchCountPages) == 6:
            searchCountPages = int("{0}{1}".format(searchCountPages[3],searchCountPages[4])) 
        else :
            searchCountPages = searchCountPages[3]  
        
        return (int(searchCountPages) // 18)
    
    def parse(self):
        browser = webdriver.Chrome(self.driverPath)
        browser.get(self.website)
        browser.maximize_window()
        
        for job in self.jobs:
            
            #if self.parser_state.is_current_job(job) == False:
            #    continue
                    
            for location in self.locations:
                
                #if self.parser_state.is_current_location(location) == False:
                #    continue
                #self.parser_state.save_state(job, location)
                
                query = "https://www.indeed.fr/jobs?q={0}&l={1}".format(job, location)
                browser.get(query)
                    
                pages_count = self._get_pages_counts(browser)
                
                for page_index in range(1, pages_count):
                    full_query = "{0}&start={1}".format(query,page_index)
                    
                    browser.get(full_query)
                    
                    items = browser.find_elements_by_xpath("//*[contains(@class,'clickcard')]")
                    dataset_len = len(self.dataset)
                    
                    for index_i,item in enumerate(items): 
                        title = item.find_element_by_xpath(".//*[contains(@class,'jobtitle')]")
                        item_link = title.get_attribute("href")
                        
                        try:
                            if (self.dataset["URL"].isin([item_link]).any()):
                                print(item_link)
                            else:
                                title, name, address, date,salaire, description, source = self.indeed_item_parser.parse(item_link)
                                self.dataset.loc[dataset_len + index_i] = [item_link, title, name, address, date, salaire, description]
                                
                                #archivage des page html
                                source_file_name = "source_file.{0}.html".format(dataset_len + index_i)
                                if not os.path.exists(source_file_name):
                                    file = open("pages/{0}".format(source_file_name), "w")
                                    source = item_link + "----------------" + source 
                                    file.write(source) 
                                    file.close() 
                                    
                                print("len(dataset)", len(self.dataset))
                                
                        except Exception as e:
                            print(e)
                            #if (len(self.dataset) == 0) or (True not in self.dataset["URL"].isin([item_link])):
                        
                        #break
                    #break
                    self.dataset.to_csv("indeed.csv",index=False)
                    #https://www.indeed.fr/jobs?q=developpeur&l=paris&start=10
                #break
            #break    
            self.dataset.drop_duplicates(subset ="URL", keep = False, inplace = True) 

In [436]:
test = IndeedPaser()
test.parse()

<class 'dict'>
len(dataset) 1773
len(dataset) 1774
len(dataset) 1775
len(dataset) 1776
https://www.indeed.fr/rc/clk?jk=925b3463f2665802&fccid=8fca4be36801c4b1&vjs=3
https://www.indeed.fr/rc/clk?jk=0afed7661a2a5af6&fccid=7077ed64aecd53a3&vjs=3
https://www.indeed.fr/rc/clk?jk=88c6253bbc63fc4e&fccid=028bbd854233c1f1&vjs=3
https://www.indeed.fr/rc/clk?jk=4ec9be2493215b5a&fccid=f15e5e4a46544874&vjs=3
len(dataset) 1777
https://www.indeed.fr/rc/clk?jk=715ef6bee0dc4d80&fccid=d51961f0c6d6c8df&vjs=3
https://www.indeed.fr/rc/clk?jk=df2af7d436d23b5d&fccid=e5e041b12c825086&vjs=3
https://www.indeed.fr/company/AID-add-intelligence-to-Data/jobs/Data-Scientist-fc87a2f33bb23599?fccid=22db5fb4d6f285e2&vjs=3
len(dataset) 1778
len(dataset) 1779
len(dataset) 1780
len(dataset) 1781
len(dataset) 1782
len(dataset) 1783
len(dataset) 1784
len(dataset) 1784
len(dataset) 1784
len(dataset) 1784
https://www.indeed.fr/rc/clk?jk=0afed7661a2a5af6&fccid=7077ed64aecd53a3&vjs=3
https://www.indeed.fr/rc/clk?jk=925b3463f266

len(dataset) 1822
len(dataset) 1823
len(dataset) 1824
len(dataset) 1824
len(dataset) 1824
https://www.indeed.fr/rc/clk?jk=925b3463f2665802&fccid=8fca4be36801c4b1&vjs=3
https://www.indeed.fr/rc/clk?jk=cdf53cf5e71884f6&fccid=feb484a8aef310be&vjs=3
https://www.indeed.fr/rc/clk?jk=4ec9be2493215b5a&fccid=f15e5e4a46544874&vjs=3
https://www.indeed.fr/rc/clk?jk=88c6253bbc63fc4e&fccid=028bbd854233c1f1&vjs=3
https://www.indeed.fr/company/ACTIVUS-Group/jobs/Data-Scientist-053959c5fc230c71?fccid=e7a47c5f5575c789&vjs=3
https://www.indeed.fr/rc/clk?jk=dd8521f428660991&fccid=237908de095b6446&vjs=3
https://www.indeed.fr/company/HPR/jobs/Data-Scientist-8d5f755b3a18ea80?fccid=3206ff697cecfd24&vjs=3
https://www.indeed.fr/company/AID-add-intelligence-to-Data/jobs/Data-Scientist-fc87a2f33bb23599?fccid=22db5fb4d6f285e2&vjs=3
https://www.indeed.fr/rc/clk?jk=715ef6bee0dc4d80&fccid=d51961f0c6d6c8df&vjs=3
len(dataset) 1825
len(dataset) 1826
len(dataset) 1827
len(dataset) 1827
len(dataset) 1827
len(dataset) 1827

len(dataset) 1854
len(dataset) 1855
len(dataset) 1856
len(dataset) 1856
len(dataset) 1856
len(dataset) 1857
https://www.indeed.fr/rc/clk?jk=b4ffdd1614d1e3ae&fccid=2301bee5373a0b4a&vjs=3
https://www.indeed.fr/company/HPR/jobs/Data-Scientist-8d5f755b3a18ea80?fccid=3206ff697cecfd24&vjs=3
https://www.indeed.fr/rc/clk?jk=df2af7d436d23b5d&fccid=e5e041b12c825086&vjs=3
https://www.indeed.fr/rc/clk?jk=e131356a90e02c3a&fccid=518f77ef3e7cf0da&vjs=3
https://www.indeed.fr/rc/clk?jk=715ef6bee0dc4d80&fccid=d51961f0c6d6c8df&vjs=3
https://www.indeed.fr/rc/clk?jk=d3859b022ec6e5ad&fccid=df6948c9b8da6236&vjs=3
https://www.indeed.fr/rc/clk?jk=90a235627582bc48&fccid=2b596b7e420f89d3&vjs=3
https://www.indeed.fr/rc/clk?jk=c9e6e768c2d2fb6a&fccid=ba550cb3a1a4f7de&vjs=3
https://www.indeed.fr/rc/clk?jk=0536e447609e0421&fccid=9b37fd62f0798dd5&vjs=3
https://www.indeed.fr/rc/clk?jk=806d52238c7b9bd4&fccid=0abb47180c61805f&vjs=3
len(dataset) 1858
len(dataset) 1859
len(dataset) 1860
len(dataset) 1860
len(dataset) 1860


len(dataset) 1878
len(dataset) 1879
len(dataset) 1880
len(dataset) 1880
len(dataset) 1880
https://www.indeed.fr/company/L'olivier-Assurances/jobs/Data-Scientist-1a544fe0f9b7abd1?fccid=c7e74c6811bdf47c&vjs=3
https://www.indeed.fr/rc/clk?jk=62422466416772ba&fccid=58db872dee6c6b74&vjs=3
https://www.indeed.fr/rc/clk?jk=1e5aa5fe73424e73&fccid=65466586cb859537&vjs=3
https://www.indeed.fr/rc/clk?jk=ca631bbf666bdb38&fccid=81fd9b0296f772b6&vjs=3
https://www.indeed.fr/rc/clk?jk=3c3715a74a7ebf22&fccid=9b246b5aaacc1ef2&vjs=3
https://www.indeed.fr/rc/clk?jk=ee132cf51c531c9a&fccid=9b6c70e895ae496c&vjs=3
https://www.indeed.fr/rc/clk?jk=304ec1611bf99c34&fccid=85ed072929053e16&vjs=3
https://www.indeed.fr/rc/clk?jk=1ea91098ccbca193&fccid=df6948c9b8da6236&vjs=3
https://www.indeed.fr/rc/clk?jk=c550e9e45ecec7c4&fccid=ff4d5564b774d59b&vjs=3
https://www.indeed.fr/company/LEC-Conseil/jobs/Data-Analyst-6849070eeae4f9de?fccid=3d3e7ac112fae637&vjs=3
len(dataset) 1881
len(dataset) 1882
len(dataset) 1883
len(datas

len(dataset) 1916
len(dataset) 1917
len(dataset) 1918
len(dataset) 1919
len(dataset) 1920
https://www.indeed.fr/rc/clk?jk=8715a4af58766e1d&fccid=159b449968878fe5&vjs=3
https://www.indeed.fr/rc/clk?jk=304ec1611bf99c34&fccid=85ed072929053e16&vjs=3
https://www.indeed.fr/rc/clk?jk=f0a64f76eeb49a07&fccid=43c5b67847a9226c&vjs=3
https://www.indeed.fr/rc/clk?jk=58b6538f2157b577&fccid=00598399eb3a4f06&vjs=3
https://www.indeed.fr/rc/clk?jk=fc6f68134131521a&fccid=50a61f19a5b28415&vjs=3
https://www.indeed.fr/rc/clk?jk=d828478b80d4c3cc&fccid=b167a1e79d3a3de2&vjs=3
https://www.indeed.fr/rc/clk?jk=ece0721c8a75904d&fccid=2a2c78431fab5d62&vjs=3
https://www.indeed.fr/rc/clk?jk=2ea4feadb4a6bff0&fccid=91e8db8cd560a49c&vjs=3
https://www.indeed.fr/rc/clk?jk=1bfc0b2c6443fe0d&fccid=9b246b5aaacc1ef2&vjs=3
https://www.indeed.fr/rc/clk?jk=4c2e1a76922820e4&fccid=cac734de0173a563&vjs=3
len(dataset) 1920
len(dataset) 1921
len(dataset) 1922
len(dataset) 1923
len(dataset) 1924
https://www.indeed.fr/rc/clk?jk=8715a4af

...

len(dataset) 3594
len(dataset) 3594
len(dataset) 3594
len(dataset) 3594
len(dataset) 3594
len(dataset) 3594
len(dataset) 3594
len(dataset) 3594
len(dataset) 3594
len(dataset) 3595
https://www.indeed.fr/rc/clk?jk=bccd1851a3843adf&fccid=abaad1bac1a84de3&vjs=3
https://www.indeed.fr/rc/clk?jk=18a43901fe92b958&fccid=a7652687ad659e9c&vjs=3
https://www.indeed.fr/rc/clk?jk=9b3f114647540706&fccid=7f7f609e4a487b6a&vjs=3
https://www.indeed.fr/company/FAYAT-IT/jobs/Chef-Projets-Business-Intelligence-F-H-0fceb8bde38f0399?fccid=a6cc4ca5444db312&vjs=3
https://www.indeed.fr/rc/clk?jk=b413245d481e906a&fccid=4e2bdf37c132734f&vjs=3
https://www.indeed.fr/rc/clk?jk=445a928042218dd1&fccid=8542430b17428de4&vjs=3
https://www.indeed.fr/rc/clk?jk=70183baf1674e5d7&fccid=4640437a3109c91f&vjs=3
https://www.indeed.fr/rc/clk?jk=6cc3fefbbe36eea4&fccid=216eb700022de6f6&vjs=3
https://www.indeed.fr/company/Nelite-France/jobs/Consultant-Skype-Business-0cc7c9f66aa78cd1?fccid=039f6d2c344f5f69&vjs=3
https://www.indeed.fr/rc

len(dataset) 3603
len(dataset) 3603
len(dataset) 3603
len(dataset) 3603
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3604
len(dataset) 3605
https://www.indeed.fr/rc/clk?jk=661dc6f054e82081&fccid=feff1881345b27f1&vjs=3
https://www.indeed.fr/rc/clk?jk=c9a3b048da49aca5&fccid=dd55f9c76eec18a9&vjs=3
https://www.indeed.fr/rc/clk?jk=9179718a1b9b3550&fccid=b252a32d4c1ba9f6&vjs=3
https://www.indeed.fr/company/Dragon-Rouge/jobs/Charg%C3%A9-D%C3%A9veloppement-New-Business-Retail-Hospitality-5bd6d8dc00abfc35?fccid=937e8e201f5e153b&vjs=3
https://www.indeed.fr/company/LAGARDERE-TRAVEL-RETAIL/jobs/Business-Analyst-0539253153788243?fccid=8ef09e65a5cc22b3&vjs=3
https://www.indeed.fr/rc/clk?jk=ad2866c544ea24cc&fccid=14835f3a1d6bbbcb&vjs=3
https://www.indeed.fr/rc/clk?jk=277b6d2791c6ee88&

len(dataset) 3612
len(dataset) 3613
https://www.indeed.fr/rc/clk?jk=c9a3b048da49aca5&fccid=dd55f9c76eec18a9&vjs=3
https://www.indeed.fr/rc/clk?jk=9179718a1b9b3550&fccid=b252a32d4c1ba9f6&vjs=3
https://www.indeed.fr/company/Dragon-Rouge/jobs/Charg%C3%A9-D%C3%A9veloppement-New-Business-Retail-Hospitality-5bd6d8dc00abfc35?fccid=937e8e201f5e153b&vjs=3
https://www.indeed.fr/company/LAGARDERE-TRAVEL-RETAIL/jobs/Business-Analyst-0539253153788243?fccid=8ef09e65a5cc22b3&vjs=3
https://www.indeed.fr/rc/clk?jk=ad2866c544ea24cc&fccid=14835f3a1d6bbbcb&vjs=3
https://www.indeed.fr/rc/clk?jk=277b6d2791c6ee88&fccid=29d37d43c382c8fe&vjs=3
https://www.indeed.fr/rc/clk?jk=db54395cf0235f69&fccid=cc5cae6bf3eb89f3&vjs=3
https://www.indeed.fr/rc/clk?jk=c917147b82b02e28&fccid=f7ba06343f7c6bb8&vjs=3
https://www.indeed.fr/rc/clk?jk=ef4254749afa18ea&fccid=84f335a45926554c&vjs=3
https://www.indeed.fr/rc/clk?jk=e61b0ef130cdf749&fccid=105ecfd0283f415f&vjs=3
len(dataset) 3613
len(dataset) 3613
len(dataset) 3613
len(dat

len(dataset) 3622
https://www.indeed.fr/rc/clk?jk=e5f9eccd67acf493&fccid=651c63598b9bd31d&vjs=3
len(dataset) 3622
len(dataset) 3622
len(dataset) 3622
len(dataset) 3622
len(dataset) 3623
https://www.indeed.fr/rc/clk?jk=93e4db06166fd5bc&fccid=a7023b829f6d269e&vjs=3
https://www.indeed.fr/rc/clk?jk=e2827fcf7b34f8e6&fccid=70bbeb2771f8c8e7&vjs=3
https://www.indeed.fr/rc/clk?jk=b413245d481e906a&fccid=4e2bdf37c132734f&vjs=3
https://www.indeed.fr/rc/clk?jk=3f1dc60248c9f029&fccid=9c88d6c072287f24&vjs=3
https://www.indeed.fr/rc/clk?jk=ad2866c544ea24cc&fccid=14835f3a1d6bbbcb&vjs=3
https://www.indeed.fr/rc/clk?jk=06d753e7d80a24f3&fccid=de640f613b6c0608&vjs=3
https://www.indeed.fr/rc/clk?jk=22cbb2b864793bc6&fccid=b3885ca934ed0066&vjs=3
https://www.indeed.fr/rc/clk?jk=b778ae3b3cc26b2e&fccid=aab0bae986cfaa7e&vjs=3
https://www.indeed.fr/rc/clk?jk=10137a317d1f5093&fccid=7ad46606e93080a6&vjs=3
https://www.indeed.fr/rc/clk?jk=e5f9eccd67acf493&fccid=651c63598b9bd31d&vjs=3
len(dataset) 3623
len(dataset) 362

KeyboardInterrupt: 

In [10]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install django

Note: you may need to restart the kernel to use updated packages.


In [403]:
class CsvToMongodbMigratorTool:
    
    def __init__(self):
        self.dataset = pd.read_csv("indeed.csv")
        self.mongodbdao = IndeedMongodbDao()
        
    def migrate(self):
        records = json.loads(self.dataset.T.to_json()).values()
        self.mongodbdao.insert_data_bulk(records)
        

In [409]:
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup

data_set = pd.read_csv("indeed.csv")
#data_set
#data_set
#for index, row in data_set.iterrows():
#    xtraSoup = BeautifulSoup(row["description"])
#    body = xtraSoup.find('body')
#    row["description"] = body.text
    
#data_set.to_csv("indeed.csv",index=False)
        
#data_set = data_set.drop(columns=['index'])
#data_set
#print(len(data_set))
#data_set.drop_duplicates(subset ="URL", keep = False, inplace = True) 
#print(len(data_set))
#data_set.to_csv("indeed.csv",index=False)
#record = data_set.iloc[1]

#indeeddao = IndeedMongodbDao()
#indeeddao.insert_data(record["URL"],record["Titre"], record["Nom entreprise"],record["Adresse"], record["Date de publication"], record["description"])


In [67]:
data_set["URL"].iloc[10]

'https://www.indeed.fr/company/LEC-Conseil/jobs/D%C3%A9veloppeur-Fullstack-F-H-ad4f8380f4e66e4a?fccid=3d3e7ac112fae637&vjs=3'

In [82]:
#migratorTool = CsvToMongodbMigratorTool()
#migratorTool.migrate()

In [437]:
from pymongo import MongoClient 
from pymongo import errors 
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError
class IndeedMongodbDao:
    def __init__(self):
        self.conn = MongoClient() 
        self.db = self.conn.Indeed
        self.collection = self.db.data
        self._create_index()
    
    def _create_index(self):
        index_name = 'url'
        if index_name not in self.collection.index_information():
            self.collection.create_index(index_name, unique=True)
    
    def _valid_url_format(self,url):
        val = URLValidator()
        try:
            val(url)
        except ValidationError as e:
            raise Exception('bad format for url {}'.format(ur))
    
    def insert_data_bulk(self,data):
        self.collection.insert_many(data)
    
    def insert_data(self, url, title, name, address, publication_date, description):
        
        try:
            if url == "":
                raise Exception('url cannot be empty {}'.format(ur))

            self._valid_url_format(url)

            if title == "":
                raise Exception('title cannot be empty {}'.format(title))

            if name == "":
                raise Exception('the name of company cannot be be empty {}'.format(title))

            if description == "":
                raise Exception('description of company cannot be be empty {}'.format(title))


            line_to_insert = {
                                "URL": url,
                                "title":title,
                                "name":name,
                                "adresse":address,
                                "publication_date":publication_date,
                                "description":description
                             }

            # Insert Data 
            result = self.collection.insert_one(line_to_insert) 
        except errors.DuplicateKeyError as e:
            print('DuplicateKeyError : url {0} aready exist'.format(url))
    
    def get_all_data(self):
        data = self.collection.find({})
        return data
    
    

In [438]:
import numpy as np 
class IndedRegexParser:
    
    def __init__(self, descriptif_text ):
        self.descriptif_text = descriptif_text
    
    def _get_items(self,pattern):   
        result_re = re.findall(pattern,self.descriptif_text)
        result = []
        for item_l_1 in result_re:
            for item_l_2 in item_l_1:
                if item_l_2 != "":
                    result.append(item_l_2)
        return result
    
    def _get_niveau_experience(self):
        #Niveau d'expérience : regex true
        result = self._get_items('([(e|E)\w+]xpérience?:?\s\d{1,15} ans?)|([(e|E)\w+]xpérience?\s?de\s?\d{1,15} ans?)|(\d{1,15} ans d\'[e|E]xpérience[s]?)')
        if len(result) == 0:
            outer = re.compile(r'Expérience:\s*.+:( (\w) (an|ans) \(((R|r)equis|(s|S)ouhaité)\))')
            m = outer.search(self.descriptif_text)
            if m is not None:
                return m.group(1)
            return []
    
    def _get_salaires(self):
        #Salaire ajusté en fonction du coût de la vie dans les différentes villes : regex true
        #return self._get_items('(\d{2,3}\s?(k|K)?)|(\d{2,3}\s?\d{3,5}\s?(k|K)?)|([(s|S)]alaire?[\s]?:?[\s]?\d{2,6}k?)|([(r|R)]émunération?[\s]?:?[\s]?\d{2,6}k?)|([(g|G)]ratification?[\s]?:?[\s]?\d{2,6}k?)') 
        #[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/an
        outer = re.compile(r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|(.*)?[\s+]?par?[\s+]?mois')
        m = outer.search(self.descriptif_text)
        if m is not None:
            return m.group(1)
        return []
            
        
    
    def get_langages_prog(self):
        #Langages de programmation requis (R, Python, Excel (VBA), SQL, C++, Java, SAS...): regex true
        return self._get_items('(R+[^\w])|([(p|P)\w+]ython?)|(Excel?)|(VBA?)|(C\+\+?)||(C?)|(Asp.net?)|(SQL?)|(NoSQL?)|(Linux?)|(MySQL?)|(MongoDB?)|(DBMaria?)|(SQL Server?)||(Java?)|(JavaScript?)|(PHP?\s?\d{1,}?)|(HTML?\s?\d{1,}?)|(CSS?\s?\d{1,}?)|(SAS?)|(C#?)|(Ruby?)|(Swift?)|(Objective-C?)|(VB.NET?)|(Kotlin?)|(Scala?)|(Bash?)|(PowerShell?)|(Shell?)|(Front End?)|(Back End?)')
    
    def get_outils_re(self):
        #Outils requis (Tableau, PowerBI...) : regex true
        return self._get_items('([(t|T)\w+]ableau?)|([(p|P)\w+]ower[(b|B)\w+][(i|I)\w+]?)|([(s|S)\w+]ymfony?\s?\d{1,}?)|([(j|J)\w+][(q|Q)\w+]uery?\s?\d{1,}?)|([(a|A)\w+]ngular?[\s?\d{1,}]?)|([(r|R)\w+]eact [(j|J)\w+][(s|S)\w+]?)|(React Native?)|(Node JS?)|([(g|G)\w+]it?)|(Visual Studio?)|(Visual Studio Code?)|(Django?)|(Flask?)|(API REST?)|(http?s?)|(HTTP?S?)')
        
        
    def get_niv_etudes_re(self):
        #Niveau d'études requis : regex true
        return self._get_items('([(f|F)\w+]ormation?\s?[(b|B)\w+]ac\s\+\s[1-8])|([(b|B)\w+]ac\s\+\s[1-8])')
        
    def get_type_cursus_re(self):
        #Type de cursus (école ingénieur, master, autodidacte...) : regex true
        return self._get_items('([(é|E)\w+]cole [(i|I)\w+]ngénieur?)|([(m|M)\w+]aster?\s?\w{3,25})|([(a|A)\w+]utodidacte?)|([(g|G)\w+]rande[s]? [(é|E)\w+]cole[s]?)|([(é|E)\w+]cole[s]? de [(c|C)\w+]ommerce[s]?)|([(i|I)\w+]ngénieur [(i|I)\w+]nformatique?)')
        
    def get_type_contrat_re(self):
        #Type de contrat : regex true
        return self._get_items('[(c|C)\w+]ontrat?:?\s\w{3,25}')
    
    def get_categorie_developpeur_re(self):
        #Grande catégorie : développeur (developeur web, dev mobile,dev) / data regex true
        return self._get_items('([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ull? [(s|S)\w+]tack)|([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ront[(\s|\-)][(e|E)\w+]nd?)|(développeu[(r|s)]e? Back[(\s|\-)][(e|E)\w+]nd?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(m|M)\w+]obile?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciel?)|([(s|S)\w+]oftware [(i|I)\w+]ngenieer?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciels?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(C|c)\w+]\+\+?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(c|C)\w+]#?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(j|J)\w+]ava?)')
    
    def get_categorie_data_re(self):
        #Grande catégorie : data regex true
        return self._get_items('([(d|D)\w+]ata [(a|A)\w+]nalyst?)|([(d|D)\w+]ata [(s|S)\w+]cientist?)|([(i|I)\w+]ngénieu[(r|s)]e? [(m|M)\w+]achine [(i|I)\w+]earning?)|([(c|C)\w+]onsultant [(b|B)\w+]usiness [(i|I)\w+]ntelligence)|([(c|C)\w+]onsultant [(d|D)\w+]ata)|([(c|C)\w+]onsultant [(b|B)\w+]ig [(d|D)\w+]ata)|([(c|C)\w+]onsultant BI)')
        

In [283]:

dataset = pd.read_csv("indeed.csv")
print(len(dataset))
#for index, row in dataset.head(1000).iterrows():
    #print(row["URL"])
    #indedRegexParser = IndedRegexParser(row["description"])
    #print(indedRegexParser._get_niveau_experience())
    #print(indedRegexParser._get_salaires())
    #print(indedRegexParser.get_outils_re())
    #print(indedRegexParser.get_niv_etudes_re())
    #print(indedRegexParser.get_type_cursus_re())
    #print(indedRegexParser.get_type_contrat_re())
    #print(indedRegexParser.get_categorie_developpeur_re())
    #print(indedRegexParser.get_categorie_data_re())


1458


In [248]:
dataset = pd.read_csv("indeed.csv")
dataset.columns

Index(['URL', 'Titre', 'Nom entreprise', 'Adresse', 'Date de publication',
       'salaire', 'description'],
      dtype='object')

In [439]:
import os
class LocalIndeedPaser:
    def __init__(self):
        self.pages_path = 'C:\\Users\\User\\Documents\\Projet_ML_gr3\\scrapping\pages'
        self.dataset = pd.read_csv("indeed.csv")
        self.salary_pattern = "[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|((.*)?[\s+]?par?[\s+]?(an|ans|mois|jour|heure))"
    
    def _get_salary(self,select_result):
        salary = ""
        for item in select_result:
            if "€" in item.text:
                outer_salary = re.compile(self.salary_pattern)
                m_salary = outer_salary.search(item.text)
                if m_salary is not None:
                    salary = m_salary.group(0)
                    break
        return salary
    
    def save_file(self):
        self.dataset.to_csv("indeed.csv",index=False)
    
    def parse(self):
        index = 0
        for root, dirs, files in os.walk(os.path.abspath(self.pages_path)):
            for file in files:
                local_file = os.path.join(root, file)
                file = open(local_file, "r") 
                html = file.read() 
                outer = re.compile(r'.*\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-')
                m = outer.search(html)
                if m is not None:
                    link = m.group(0).replace("----------------","")
                    dataset_index = self.dataset[self.dataset["URL"] == link].index.values.astype(int)
                    #print(self.dataset["salaire"][dataset_index].values[0])
                    salary = self.dataset["salaire"][dataset_index]
                    
                    if (len(salary.values) == 0):
                        continue 
                        
                    if (salary.values[0] != "-"):
                        continue
                        
                    html = html.replace(m.group(0),"")
                    soup = BeautifulSoup(html, "html.parser")
                    result = soup.select(".jobsearch-JobMetadataHeader-item")
                    if len(result) > 0:
                        self.dataset["salaire"][dataset_index] = self._get_salary(result)
                        continue
                        
                    result = soup.select(".jobsearch-JobMetadataHeader-itemWithIcon .jobsearch-JobMetadataHeader-iconLabel")
                    if len(result) > 0:
                        self.dataset["salaire"][dataset_index] = self._get_salary(result)
                        continue
                    
                    result = soup.select("#jobDescriptionText")
                    if len(result) > 0:
                        index = index + 1 
                        outer_salary = re.compile(self.salary_pattern)
                        m_salary = outer_salary.search(result[0].text)
                        if m_salary is not None:
                            salary = m_salary.group(0)
                            print(salary)
                            break
                    
                    
                    

In [440]:
localIndeedPaser = LocalIndeedPaser()
localIndeedPaser.parse()

40 experts triés sur le volet150 méthodes autour de la performance commerciale, performance marketing et expérience client200 clients dans tous les secteurs d’activité5000 sessions de travail par an


In [429]:
localIndeedPaser.save_file()

In [430]:
xx = pd.read_csv("indeed.csv")
xx.iloc[5]["URL"]

'https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0Afb2xqmmvnLeU-eGdBKKKG4nWVW1VrA-McngqAoir6deY3vbRrCKR3mm24AFrxzAqh7I0b4RySssf7nCnyJKwdLxN7QMXQhkNsCK8XusozsvmW1eF3sglEZHhdIQHVv2RHiak76E1rzsanbeeCr-cV9qVWesmcumaHAoMbOeEA4CHTqraW36OrylaxzxDaGwF_dfKkQLBx0PCe4_Ff_lSISpdlsfcx6xvq4BZ77T4OvqwxWuJAMl11doXmHjNHuVfMkBP1COQrS9rugwU9E7eEdahFZj9oBzpewwcOVsBIqbEAhkX0Ys22r01XhfaozrQVMtXwdb567ld5AqNVyikTl10F-fkscayOe1CvGQ4IlwT_rWjXaG_9AuaL5eSevzwbi83GyKGC7uU0NkdsS28ts1fuPjxBL2JPI44-dRVWbi3rjwIsExHL3ixrlh9Zsgs=&p=5&fvj=1&vjs=3'

In [431]:
xx

,URL,Titre,Nom entreprise,Adresse,Date de publication,salaire,description
0,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Sup3r Développeur Web Full Stack @Startup,GlobeSailor SAS,Paris (75),2019-10-03 03:11:00.197236,NaN,4pp31 4 T0u5 135 D3v310p3r5 w3b full st4k !\nP...
1,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Informatique H/F,VALENTE SECURYSTAR,94320 Thiais,2019-10-02 03:11:14.353355,1 850 € par mois,"VALENTE SECURYSTAR, fabricant de portes blindé..."
2,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Android (H/F),Universign,Paris (75),2019-09-04 03:11:27.090785,NaN,Universign est le pure player de référence sur...
3,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Ionic / Firebase,NEXTT,92200 Neuilly-sur-Seine,2019-09-19 03:11:39.836685,NaN,Dans le cadre du développement d'une applicati...
4,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Data Engineer / Développeur NLP H/F,Wolters Kluwer,Paris (75),2019-09-04 03:11:53.014114,NaN,"Au sein du Pôle Droit et Réglementation, ratta..."
5,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Ratp Smart Systems,Paris (75),2019-09-10 03:12:05.948019,NaN,1er jet :\nPour accélérer sa transformation et...
6,https://www.indeed.fr/rc/clk?jk=6986aa9ce3ca25...,Développeur iOS (H/F),Universign,Paris (75),2019-10-01 03:12:18.742210,-,Universign est le pure player de référence sur...
7,https://www.indeed.fr/rc/clk?jk=8ee618fc0da425...,BALENCIAGA - Coordinateur/trice Développement ...,Groupe Kering,Paris (75),2019-10-03 16:12:31.688147,-,Summary\nFondée en 1917 par l’Espagnol Cristób...
8,https://www.indeed.fr/rc/clk?jk=2f40c0f1d86b7b...,Développeur Junior (ReactJS / Node.js),AKOYA,Paris (75),2019-10-01 03:12:44.296664,NaN,AKOYA GENIUS\nEn complément de ses activités d...
9,https://www.indeed.fr/rc/clk?jk=fdb86ccd993a5c...,BALENCIAGA - Coordinateur/trice Développement ...,Balenciaga,Paris (75),2019-10-03 15:12:57.009145,-,Summary\nFondée en 1917 par l’Espagnol Cristób...
